<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import openai
from openai import OpenAI

import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: f398f57a1163e8aae0ec0bd88c200edc599ffc4a

matplotlib: 3.8.0
termcolor : 2.4.0
json      : 2.0.9
tqdm      : 4.66.4
openai    : 1.30.5
numpy     : 1.26.4
watermark : 2.4.3
pandas    : 2.1.4



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

Then we are ready to instantiate the client

In [4]:
client = OpenAI()

We start by getting a list of supported models.

In [5]:
model_list = json.loads(client.models.list().json())["data"]

In total we have 33 models

In [6]:
len(model_list)

32

Along with some information about each model...

In [7]:
model_list[:3]

[{'id': 'dall-e-3',
  'created': 1698785189,
  'object': 'model',
  'owned_by': 'system'},
 {'id': 'whisper-1',
  'created': 1677532384,
  'object': 'model',
  'owned_by': 'openai-internal'},
 {'id': 'gpt-4-turbo-2024-04-09',
  'created': 1712601677,
  'object': 'model',
  'owned_by': 'system'}]

But let's just get a list of model names

In [8]:
print("\n".join(sorted([model["id"] for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini
gpt-4o-mini-2024-07-18
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [9]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

Which produces a response object

In [10]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a named tuple

The model answer can be found in the "message" dictionary inside the "choices" list

In [11]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's weakness is Kryptonite, a radioactive mineral from his home planet of Krypton. Exposure to Kryptonite weakens Superman and can potentially kill him.", role='assistant', function_call=None, tool_calls=None))

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [12]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 5.68 ms, sys: 1.32 ms, total: 6.99 ms
Wall time: 4.65 s


And we can access each of the answers individually int he choices list

In [13]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
There are several different kinds of Kryptonite in the DC Comics universe, each with different effects on Superman and other Kryptonians:

1. Green Kryptonite: The most well-known type of Kryptonite, green Kryptonite weakens Superman and other Kryptonians, and prolonged exposure can be fatal.

2. Red Kryptonite: Red Kryptonite has unpredictable and temporary effects on Kryptonians, causing them to act irrationally or strangely.

3. Blue Kryptonite: Blue Kryptonite has similar effects to green Kryptonite, but is specifically harmful to Bizarro, a flawed duplicate of Superman.

4. Gold Kryptonite: Gold Kryptonite permanently removes a Kryptonian's superpowers.

5. Black Kryptonite: Black Kryptonite can split a Kryptonian into two separate beings, one good and one evil.

6. White Kryptonite: White Kryptonite is lethal to plant life, but has no effect on Kryptonians.

7. Anti-Kryptonite: Anti-Kryptonite weakens non-Kryptonians and can be used as a weapon against Superman's enemie

In [14]:
response.usage

CompletionUsage(completion_tokens=780, prompt_tokens=17, total_tokens=797)

# Temperature

In [15]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=1.5
)

CPU times: user 4.78 ms, sys: 1.32 ms, total: 6.1 ms
Wall time: 3.29 s


In [16]:
print(response.choices[0].message.content)

Once upon a time, in a small town tucked away in the mountains, there lived a young girl named Lily. Lily was known for her kind heart and insatiable curiosity. One day, as she was exploring the woods beyond her village, she stumbled upon a fairy who was trapped under a fallen tree branch.

Lily all but forgot her fear as she rushed to the fairy's side, trying to lift the heavy branch. With all her might, she pushed and pulled until the branch finally lifted, setting the fairy free.

Overcome with gratitude, the fairy granted Lily a single wish. Without hesitation, Lily wished for happiness for all the villagers in her town. The fairy smiled before disappearing in a cloud of shimmering dust.

From that day on, the town was filled with joy and harmony. Lily was hailed as a hero, and everyone lived happily ever after. And though she never saw another fairy again, Lily knew that her small act of kindness had made a big difference in the lives of those she loved.


In [17]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 5.14 ms, sys: 1.35 ms, total: 6.49 ms
Wall time: 2.8 s


In [18]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lily. Lily was known throughout the village for her kindness and generosity. She would often spend her days helping the elderly, feeding stray animals, and planting flowers in the village square.

One day, a terrible storm hit the village, causing widespread damage and leaving many families homeless. Despite the devastation, Lily remained determined to help those in need. She gathered her friends and together they worked tirelessly to rebuild homes, provide food and clothing, and offer comfort to those who had lost everything.

As word of Lily's selfless acts spread, people from neighboring villages came to offer their support. The once divided communities now came together in unity, inspired by Lily's compassion and dedication.

In the end, the village was restored to its former glory, and the people were forever grateful to Lily for showing them the power of kindness and the importance of co

# Function Calls

In [19]:
def chat(messages, functions):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        # Define the functions the model is allowed to use
        functions=functions
    )
    
    return response

In [20]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        print(message)
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "assistant" and message['function_call']:
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message['role']]))
        elif message["role"] == "assistant" and not message['function_call']:
            print(colored(f"assistant: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "function":
            print(colored(f"function ({message.name}): {message.content}\n", role_to_color[message.role]))


Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [21]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [22]:
messages = []

messages.append(
    {"role": "system", 
     "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    })

messages.append(
    {"role": "user", 
     "content": "What's the weather like today"
    })

In [23]:
chat_response = chat(messages, functions=functions)
assistant_message = chat_response.choices[0].message
messages.append({
 "role":  assistant_message.role,
 "content":  assistant_message.content,
 "function_call":  assistant_message.function_call,
})

In [24]:
pretty_print_conversation(messages)

{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

{'role': 'user', 'content': "What's the weather like today"}
user: What's the weather like today

{'role': 'assistant', 'content': 'Sure, I can help with that. Could you please provide me with your current location (city, state)?', 'function_call': None}
assistant: Sure, I can help with that. Could you please provide me with your current location (city, state)?



Once we provide the missing information, it will generate the appropriate function arguments for us.

In [25]:
messages.append(
    {"role": "user", 
     "content": "I'm in New York, NY."
    })

In [26]:
chat_response = chat(messages, functions=functions)
assistant_message = chat_response.choices[0].message
messages.append({
 "role":  assistant_message.role,
 "content":  assistant_message.content,
 "function_call":  assistant_message.function_call,
})

In [27]:
pretty_print_conversation(messages)

{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

{'role': 'user', 'content': "What's the weather like today"}
user: What's the weather like today

{'role': 'assistant', 'content': 'Sure, I can help with that. Could you please provide me with your current location (city, state)?', 'function_call': None}
assistant: Sure, I can help with that. Could you please provide me with your current location (city, state)?

{'role': 'user', 'content': "I'm in New York, NY."}
user: I'm in New York, NY.

{'role': 'assistant', 'content': None, 'function_call': FunctionCall(arguments='{"location":"New York, NY","format":"celsius"}', name='get_current_weather')}
assistant: FunctionCall(arguments='{"location":"New York, NY","format":"celsius"}', name='get_current_weather')



## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [28]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

This last-minute change means we can't spend excessive time on the client's project.


# Formatted output

In [29]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = client.chat.completions.create(
          model = "gpt-3.5-turbo",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 8.25 ms, sys: 2.55 ms, total: 10.8 ms
Wall time: 3.48 s


In [30]:
json_output = response.choices[0].message.content

In [31]:
output = json.loads(json_output)

In [32]:
output["ingredients"]

['1 cup all-purpose flour',
 '2 tablespoons sugar',
 '1 teaspoon baking powder',
 '1/2 teaspoon baking soda',
 '1/4 teaspoon salt',
 '1 cup buttermilk',
 '1 large egg',
 '2 tablespoons melted butter',
 '1 cup fresh blueberries']

In [33]:
output["instructions"]

['In a large bowl, combine the flour, sugar, baking powder, baking soda, and salt.',
 'In a separate bowl, whisk together the buttermilk, egg, and melted butter.',
 'Pour the wet ingredients into the dry ingredients and mix until just combined.',
 'Gently fold in the fresh blueberries.',
 'Heat a non-stick skillet or griddle over medium heat and lightly grease with butter or cooking spray.',
 'Pour 1/4 cup of the batter onto the skillet for each pancake.',
 'Cook until bubbles form on the surface of the pancake, then flip and cook for an additional 1-2 minutes until golden brown.',
 'Repeat with the remaining batter.',
 'Serve the blueberry pancakes warm with maple syrup and additional fresh blueberries.']

# Translation

In [34]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [35]:
response.choices[0].message.content

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [36]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [37]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [38]:
print(response.choices[0].message.content)

Fruit,Color,Flavor
neoskizzles,purple,candy
loheckles,grayish blue,tart
pounits,bright green,savory
loopnovas,neon pink,cotton candy
glowls,pale orange,sour


In [39]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [40]:
print(response.choices[0].message.content)

Pounits are bright green fruits that are more savory than sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>